In [1]:
import glob
import os
import json
import xml.etree.ElementTree as et
import soundfile as sf
import numpy as np
from pathlib import Path
from tqdm import tqdm
import argparse
import itertools
import wave
from pydub import AudioSegment

/opt/anaconda3/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
a_path = "/Users/ccui/Desktop/AMI_sample/clips_monoSpk/ES2002a/A/sample1/A_Array1-01.wav"
b_path = "/Users/ccui/Desktop/AMI_sample/clips_monoSpk/ES2002a/B/sample1/B_Array1-01.wav"

In [14]:
infiles = [a_path, b_path]
outfile = "/Users/ccui/Desktop/AMI_sample/clips_synthesis/ab_1.wav"

data= []
for infile in infiles:
    w = wave.open(infile, 'rb')
    data.append( [w.getparams(), w.readframes(w.getnframes())] )
    w.close()

output = wave.open(outfile, 'wb')
output.setparams(data[0][0])
output.writeframes(data[0][1])
output.writeframes(data[1][1])
output.close()

In [5]:
a = AudioSegment.from_file(
    a_path,
    format="wav",
    frame_rate=16000)
b = AudioSegment.from_file(b_path, format="wav")

In [6]:
new = a+b
new.export("/Users/ccui/Desktop/AMI_sample/clips_synthesis/ab_1.wav",format="wav")

<_io.BufferedRandom name='/Users/ccui/Desktop/AMI_sample/clips_synthesis/ab_1.wav'>

In [18]:
array, fs = sf.read(
    "/Users/ccui/Desktop/AMI_sample/clips_synthesis/ab_1.wav",
    dtype="float32",
)

a, fs = sf.read(
   a_path,
    dtype="float32",
)
b, fs = sf.read(
   b_path,
    dtype="float32",
)

In [19]:
print((a+b).shape)

(64000,)


In [22]:
print(len(a.shape))

1


In [20]:
sf.write(
    "/Users/ccui/Desktop/AMI_sample/clips_synthesis/ab_1.wav",
    a+b,
    fs,
)


In [21]:
l = ["A","B","C","D"]
print(list(itertools.combinations(l, 2)))

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]


In [2]:
in_dir = "/Users/ccui/Desktop/AMI_sample/clips_monoSpk"
out_dir = "/Users/ccui/Desktop/AMI_sample/clips_synthesis"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

folder_list = glob.glob(os.path.join(in_dir, "*"))
for meeting in folder_list:
    meeting_name = Path(meeting).stem.split(".")[-1]
    spk_list = glob.glob(os.path.join(meeting, "*"))
    # Combine speakers in pairs
    spk_comb = list(itertools.combinations(spk_list, 2))
    sample = 1
    for couple in spk_comb:
        spk1 = couple[0]
        spk2 = couple[1]
        # Get all samples for each speaker
        spk1_samples = glob.glob(os.path.join(spk1, "*"))
        spk2_samples = glob.glob(os.path.join(spk2, "*"))
        # Combine samples in pairs
        couple_com_list = list(itertools.product(spk1_samples,spk2_samples))
        for couple_com in couple_com_list:
            # Each sample has multiple arrays
            arrays_spk1 = glob.glob(os.path.join(couple_com[0], "*"))
            arrays_spk2 = glob.glob(os.path.join(couple_com[1], "*"))
            mic_num = 1
            # Synthesis two different sets of speakers with the same array number
            for arr_spk1 in arrays_spk1:
                arr_num_spk1 = Path(arr_spk1).stem.split(".")[-1].split("_")[-1]
                for arr_spk2 in arrays_spk2:
                    arr_num_spk2 = Path(arr_spk2).stem.split(".")[-1].split("_")[-1]
                    if arr_num_spk1 == arr_num_spk2:
                        mic = "mic"+str(mic_num)
                        sample_folder = os.path.join(out_dir, meeting_name,"sample"+str(sample))
                        if not os.path.exists(sample_folder):
                            os.makedirs(sample_folder)
                        spk1_file_name = os.path.join(sample_folder,
                                                      "spk1_" + mic+".wav")
                        cmd_cp_spk1 = f"cp -f {arr_spk1} {spk1_file_name}"
                        os.system(cmd_cp_spk1)
                        spk2_file_name = os.path.join(
                            sample_folder, "spk2_" + mic + ".wav")
                        cmd_cp_spk2 = f"cp -f {arr_spk2} {spk2_file_name}"
                        os.system(cmd_cp_spk2)
                        mixture_file_name = os.path.join(
                            sample_folder, "mixture_" + mic + ".wav")

                        spk1_wav, fs = sf.read(
                            spk1_file_name,
                            dtype="float32",
                        )
                        spk2_wav, fs = sf.read(
                            spk2_file_name,
                            dtype="float32",
                        )
                        sf.write(
                            mixture_file_name,
                            spk1_wav + spk2_wav,
                            fs,
                        )
                        mic_num+=1
            sample+=1
        # break


        # print(len(spk2_samples))
    # print(spk_comb)
    break